In [1]:
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.elasticsearch:elasticsearch-spark-30_2.12:8.13.2,org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.spark:spark-avro_2.12:3.3.1 pyspark-shell'

In [2]:
import io
import os
import pyspark.sql.types as T
from pyspark.sql import SparkSession, functions as func

In [3]:
spark = SparkSession.builder \
            .master('local[*]') \
            .appName('coincap-consumer') \
            .getOrCreate()

24/04/22 19:56:57 WARN Utils: Your hostname, Oluseguns-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.109 instead (on interface en0)
24/04/22 19:56:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/iamraphson/.ivy2/cache
The jars for the packages stored in: /Users/iamraphson/.ivy2/jars
org.elasticsearch#elasticsearch-spark-30_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5f6c548e-f69e-4c13-9953-d693f99fea4f;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/homebrew/Cellar/apache-spark/3.5.1/libexec/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.elasticsearch#elasticsearch-spark-30_2.12;8.13.2 in central
	found org.scala-lang#scala-reflect;2.12.8 in central
	found org.slf4j#slf4j-api;1.7.6 in central
	found commons-logging#commons-logging;1.1.1 in central
	found javax.xml.bind#jaxb-api;2.3.1 in central
	found com.google.protobuf#protobuf-java;2.5.0 in central
	found org.apache.spark#spark-yarn_2.12;3.3.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr

In [4]:
df_raw_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", 'localhost:9092') \
    .option("subscribe", 'data.asset_prices_plain') \
    .option("startingOffsets", "latest") \
    .load()

In [5]:
df_value_stream = df_raw_stream.selectExpr('CAST(value AS STRING)')

In [6]:
json_schema = T.StructType([
    T.StructField("asset_name", T.StringType()),
    T.StructField('asset_price', T.StringType()),
    T.StructField('collected_at', T.StringType()),
])

df_value_stream = df_value_stream \
    .withColumn('decoded_data', func.from_json(func.col('value'), json_schema).alias("data")) \
    .select('decoded_data.*')

print(df_value_stream.printSchema())

root
 |-- asset_name: string (nullable = true)
 |-- asset_price: string (nullable = true)
 |-- collected_at: string (nullable = true)

None


In [7]:
def sink_console(df_stream, output_mode: str = 'complete', processing_time: str = '5 seconds'):
    df_stream.writeStream \
        .outputMode(output_mode) \
        .trigger(processingTime=processing_time) \
        .format("console") \
        .option("truncate", False) \
        .start()

def sink_es(df_stream, output_mode: str = 'complete'):
    df_stream.writeStream \
        .outputMode(output_mode) \
        .format("org.elasticsearch.spark.sql") \
        .option("checkpointLocation", "checkpoint_location") \
        .option("es.nodes", "localhost") \
        .option("es.port", "9200") \
        .option("es.index.auto.create", "true") \
        .option("es.nodes.wan.only", "true") \
        .option("es.resource", "assets/_doc") \
        .start()

In [8]:
#sink_console(df_value_stream, output_mode='append')
sink_es(df_value_stream, output_mode='append')

24/04/22 19:57:03 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/22 19:57:04 ERROR Executor: Exception in task 0.0 in stage 0.0 (TID 0)
java.lang.NoSuchMethodError: 'org.apache.spark.sql.catalyst.encoders.ExpressionEncoder org.apache.spark.sql.catalyst.encoders.RowEncoder$.apply(org.apache.spark.sql.types.StructType)'
	at org.elasticsearch.spark.sql.streaming.EsStreamQueryWriter.<init>(EsStreamQueryWriter.scala:50)
	at org.elasticsearch.spark.sql.streaming.EsSparkSqlStreamingSink.$anonfun$addBatch$5(EsSparkSqlStreamingSink.scala:72)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:93)
	at org.apache.spark.TaskContext.runTaskWithListeners(TaskContext.scala:166)
	at org.apache.spark.scheduler.Task.run(Task.scala:141)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:620)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally(SparkErrorUtils.scala:64)


In [9]:
spark.stop()

24/04/22 19:35:05 ERROR MicroBatchExecution: Query [id = 87ff0aac-024b-4849-a9df-016d428f310b, runId = babf3f71-5804-45a4-b44a-2a72678662a4] terminated with error
org.apache.spark.SparkException: [INTERNAL_ERROR] The Spark SQL phase planning failed with an internal error. You hit a bug in Spark or the Spark plugins you use. Please, report this bug to the corresponding communities or vendors, and provide the full stack trace.
	at org.apache.spark.SparkException$.internalError(SparkException.scala:107)
	at org.apache.spark.sql.execution.QueryExecution$.toInternalError(QueryExecution.scala:536)
	at org.apache.spark.sql.execution.QueryExecution$.withInternalError(QueryExecution.scala:548)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:219)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:900)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:218)
	at org.apache.spark.sql.execution.QueryExec